# Inferencia de nuevas perovskitas

En este notebook se utilizará la Red Neuronal Artificial para inferir nuevos compuestos que puedan cristalizar en la perovskita de mayor simetría (el aristotipo). Esta perovskita corresponde al grupo espacial no. 221.

Vamos a comenzar por cargar todas las librerías necesarias. Ejecuta el siguiente recuadro.

In [1]:
import itertools as it
import pandas as pd
import numpy as np
import grow_crystal 
import matplotlib.pyplot as plt
import neighdist
import patolli
import keras.models as models

ModuleNotFoundError: No module named 'pymatgen'

Ahora cargaremos el archivo datosrahm.csv, que contiene el radio atómico y la electronegatividad de Pauling de cada elemento.

In [4]:
datos = pd.read_csv('datosrahm.csv')

maindict = {}
for row in range(datos.shape[0]):
    maindict[datos['Symbol'][row]] = \
    datos.iloc[row,:][['elecPau','atradrahm']].values

Al ejecutar el recuadro anterior, se crea un diccionario de Python con el nombre de <i>maindict</i>. Ese diccionario tiene como 'keys' a los elementos y como 'values' un array con la electronegatividad y el radio atómico, en ese orden.

Ahora cargaremos el mejor modelo (archivo.h5) y el diccionario que contiene los promedios y las desviaciones estándar de cada rasgo (archivo.npy). Si todavía no están esos archivos en esta carpeta, cópialos ahora.

In [5]:
model_dict = np.load('feature_standarisation.npy').item()
model = models.load_model('best_model.h5')

NameError: name 'models' is not defined

In [6]:
#Esto es un resumen del modelo entrenado. Para más información, ve a https://keras.io/models/about-keras-models/
model.summary()

NameError: name 'model' is not defined

Debido a que se modelaran perovskitas cúbicas, necesitamos definir la posición de los átomos dentro la celda unitaria con los sitios de Wyckoff ocupados. En la perovskita cúbica ideal, éstos son:

In [7]:
wyckcub = {0:{'A' : np.asarray([[0.0,0.0,0.0]])},
          1:{'B' : np.asarray([[0.5,0.5,0.5]])},
          2:{'anion' : np.asarray([[0.5,0.5,0.0],[0.5,0.0,0.5],[0.0,0.5,0.5]])}}

<b> NO MODIFIQUES LOS VALORES DEL RECUADRO ANTERIOR </b>

Los átomos A son los átomos con geometría dodecaédrica, aquellos B se ubican en sitios con geometría octaédrica. Los aniones conectan a los octaedros mediante la compartición de vértices.

Ahora hay que definir un parámetro de red con el que vamos a probar.

In [12]:
#Cuando quieras, modifica el valor que aquí se declara.
lattice_parameter = 3.795

In [9]:
#Esta es la distancia de corte de la función de ambientes químicos. No la modifiques.
dist = 25

A continuación, define qué elementos ocuparán los sitios A, B y X:

In [13]:
#Define a los elementos dentro de los ''
A = 'Ca'
B = 'Ti'
anion = 'O'

In [14]:
#Con este recuadro, se guardan las posiciones de los átomos dentro de la celda unitaria en un archivo .xyz
positions = lattice_parameter*np.concatenate((wyckcub[0]['A'],wyckcub[1]['B'], wyckcub[2]['anion']),axis=0)
elements = [A,B,anion,anion,anion]

df = pd.DataFrame({'element':elements, 'x':positions[:,0], 'y':positions[:,1],'z':positions[:,2]})

with open(A+B+anion+'3.xyz','w') as f:
    f.write(str(len(df))+ '\n'+'\n')
    for row in range(df.shape[0]):
        f.write(df['element'][row] + '    ' + "%.4f" % np.round(df['x'][row],4) + '    '  + \
                "%.4f" % np.round(df['y'][row],4) + '    ' + "%.4f" % np.round(df['z'][row],4) + '\n')


In [15]:
df

,element,x,y,z
0,Ca,0.0000,0.0000,0.0000
1,Ti,1.8975,1.8975,1.8975
2,O,1.8975,1.8975,0.0000
3,O,1.8975,0.0000,1.8975
4,O,0.0000,1.8975,1.8975


Vamos a obtener los rasgos en bruto (raw_features) para posteriormente calcular los rasgos geométricos y de empaquetamiento y las funciones de ambiente químico.

In [ ]:
def get_raw_features(elements = [['elemento_A'],['elemento_B'],['elemento_anion']]):
    compositions = [[1.0], [1.0],[1.0]]
    multiplicities = [[1],[1],[3]]
    raw_features = [np.dot(np.asarray(subind), np.asarray([maindict.get(item,None) \
              for item in site])) for site, subind in zip (elements, compositions)]

    raw_features = np.asarray(raw_features)
    raw_features = np.concatenate((np.zeros((1,raw_features.shape[1])),raw_features), axis = 0)
    
    return raw_features

In [ ]:
def compute_local_functions(raw_features = np.zeros((4,2))):
    radii = raw_features[:,1]
    elec = raw_features[:,0]

    elec = elec.reshape((4,1))
    delec = np.repeat(elec[:,np.newaxis],4,axis=2) - np.repeat(elec[:,np.newaxis],4,axis=2).T
    delec = delec.reshape((delec.shape[0],delec.shape[2]))
    fr = np.zeros((1,4,4-1))

    p, z, n, m = neighdist.positions(pos = wyckcub, angles = [90,90,90], abc = [lattice_parameter,]*3, dist = dist)
    r = neighdist.rij(mult=m,p=p,zero=z, dist=dist, radii = np.ravel(radii))

    temp = np.multiply(r,delec)
    temp = temp[~np.eye(temp.shape[0], dtype=bool)].reshape(temp.shape[0],-1)
    fr[0] = temp

    fr = fr.reshape((fr.shape[0],1,fr.shape[1]*fr.shape[2]))
    
    return fr

In [ ]:
raw_features = get_raw_features(elements = [[A],[B],[anion]])

In [ ]:
print(raw_features)
print('El número de sitios totales en la descripción de cada compuesto:', raw_features.shape[0])
print('La primera columna contiene a la electronegatividad y la segunda al radio atómico promedio de cada sitio, en ese orden:')

In [ ]:
quot = patolli.compute_quotients(X = raw_features[np.newaxis,:,:])
local_functions = compute_local_functions(raw_features=raw_features)

Ahora se concatenan los rasgos de empaquetamiento y las funciones de localidad.

In [ ]:
x = np.concatenate((quot,local_functions), axis=2)

A continuación, se estandarizan los datos de entrada del compuesto propuesto:

In [ ]:
X = (x - model_dict['mean'])/model_dict['std']

Y se efectúa la inferencia enseguida:

In [ ]:
y = model.predict(X[:,0,:])

In [ ]:
print('El compuesto, de formula ' + A + B + anion + '3 y con el parametro de red ' + str(lattice_parameter) + ' angstroms,' +'\n' + \
      '   tiene una probabilidad de cristalizar en perovskita cúbica de ' + str(y))